# Tipologia de hogar

In [1]:
#Librerias necesarias para el procesamiento y visualización de la información
import pandas as pd 
import pyreadstat
from tqdm import tqdm

# 1 Implementacion de función para determinar la tipologia

#### 1.1 Carga de data

In [2]:
# Importamos un archivo excel de prueba que contiene las primeras 102 filas del censo 2010
df_prueba = pd.read_excel('base_censo_2010/Data_prueba.xlsx')

In [3]:
# Analizamos la base sus identificadores,  el total de filas y columnas del DataFrame
print(df_prueba.shape)
df_prueba.head()

(102, 21)


,I01,I02,I03,I04,I05,I06,I09,I10,URP,P00,...,P01,P02,P03,P34,P37,id_viv,id_hog,id_provin,id_can,id_parr
0,Azuay,1,50,1,1,1,1,1,Area urbana,1,...,Hombre,1,42,Casado/a,NaN,1015000100101000,11,AZUAY,CUENCA,CUENCA
1,Azuay,1,50,1,1,1,1,1,Area urbana,2,...,Mujer,2,40,Casado/a,3.0,1015000100101000,11,AZUAY,CUENCA,CUENCA
2,Azuay,1,50,1,1,1,1,1,Area urbana,3,...,Mujer,3,16,Soltero/a,NaN,1015000100101000,11,AZUAY,CUENCA,CUENCA
3,Azuay,1,50,1,1,1,1,1,Area urbana,4,...,Mujer,3,11,NaN,NaN,1015000100101000,11,AZUAY,CUENCA,CUENCA
4,Azuay,1,50,1,1,1,1,1,Area urbana,5,...,Mujer,3,2,NaN,NaN,1015000100101000,11,AZUAY,CUENCA,CUENCA


In [4]:
# lista de columnas del DataFrame
df_prueba.columns

Index(['I01', 'I02', 'I03', 'I04', 'I05', 'I06', 'I09', 'I10', 'URP', 'P00',
       'PCD', 'P01', 'P02', 'P03', 'P34', 'P37', 'id_viv', 'id_hog',
       'id_provin', 'id_can', 'id_parr'],
      dtype='object')

#### 1.2 Preparar la data con groupby

In [5]:
# Conversion de tipo de datos string
df_prueba['P02'] = df_prueba['P02'].astype(str)

In [6]:
# Agrupamiento por identificador de hogar y relacion con jefe de hogar
data_agrupada = df_prueba.groupby(["id_hog", 'P02'])

In [7]:
# tamaño de cada tipo de categoria en variable P02
data_agrupada.size()

id_hog  P02
11      1      1
        2      1
        3      3
21      1      1
        3      1
              ..
261     5      2
271     1      1
281     1      1
        2      1
        3      2
Length: 72, dtype: int64

In [8]:
# Conteo de categorias de relacion con respecto al jefe de hogar
df_conteo = data_agrupada.size()

In [9]:
df_conteo

id_hog  P02
11      1      1
        2      1
        3      3
21      1      1
        3      1
              ..
261     5      2
271     1      1
281     1      1
        2      1
        3      2
Length: 72, dtype: int64

In [10]:
# Asignar este conteo al dataframe en una nueva columna
df_resultante = df_conteo.reset_index(name='Cuenta de parentesco o relación tiene con el/la jefe/a del hogar')

In [11]:
df_resultante

,id_hog,P02,Cuenta de parentesco o relación tiene con el/la jefe/a del hogar
0,11,1,1
1,11,2,1
2,11,3,3
3,21,1,1
4,21,3,1
...,...,...,...
67,261,5,2
68,271,1,1
69,281,1,1
70,281,2,1


#### 1.3 Codificacion por tipo de hogar

In [12]:
# Convertir a codificacion las categorias generadas en filas
df_resultante["Codigo Hogar"] = df_resultante.groupby(["id_hog"])['P02'].transform(lambda x : ''.join(x))

In [13]:
df_resultante

,id_hog,P02,Cuenta de parentesco o relación tiene con el/la jefe/a del hogar,Codigo Hogar
0,11,1,1,123
1,11,2,1,123
2,11,3,3,123
3,21,1,1,13
4,21,3,1,13
...,...,...,...,...
67,261,5,2,15
68,271,1,1,1
69,281,1,1,123
70,281,2,1,123


#### 1.4 Conteo por cantidad de hijos de hombres y mujeres

In [14]:
# Agrupamiento por identificador de hogar y sexo 
data_hijos = df_prueba.groupby(["id_hog",'P01','P02'])
# tamaño de cada tipo de categoria en variable P01
df_conteo_hijos = data_hijos.size()
df_hijos = df_conteo_hijos.reset_index(name='Cuenta genero')
df_genero_hijos = df_hijos[(df_hijos['P02']=='3')]
df_conteo_hijos_resultante=pd.pivot_table(df_genero_hijos,
                                          index='id_hog',
                                          columns='P01',
                                          values='Cuenta genero')

In [15]:
df_conteo_hijos_resultante

P01,Hombre,Mujer
id_hog,,
11,NaN,3.0
21,1.0,NaN
31,3.0,2.0
41,2.0,3.0
71,1.0,1.0
81,1.0,2.0
101,1.0,2.0
111,1.0,NaN
121,NaN,1.0


#### 1.4 Conteo por cantidad de parientes

In [16]:
data_parientes = df_prueba.groupby(["id_hog",'P02'])
conteo_parientes = data_parientes.size()
df_parientes =conteo_parientes.reset_index(name='Cuenta')
df_parientes = df_parientes[(df_parientes['P02']=='4')|(df_parientes['P02']=='5')|(df_parientes['P02']=='6')|(df_parientes['P02']=='7')]
df_parientes['Parientes_resultantes'] ='Parientes'
df_parientes_resultante = pd.pivot_table(df_parientes,
                                index = 'id_hog', 
                                columns = "Parientes_resultantes" , 
                                values = "Cuenta", 
                                aggfunc = "sum")

In [17]:
df_parientes_resultante

Parientes_resultantes,Parientes
id_hog,
31,2
41,1
81,3
131,1
171,1
181,1
201,1
211,1
221,2


In [18]:
data_no_parientes = df_prueba.groupby(["id_hog",'P02'])
conteo_no_parientes = data_no_parientes.size()
df_no_parientes =conteo_no_parientes.reset_index(name='Cuenta')
df_no_parientes = df_no_parientes[(df_no_parientes['P02']=='8')|(df_no_parientes['P02']=='9')|(df_no_parientes['P02']=='10')|(df_no_parientes['P02']=='11')]
df_no_parientes['No_Parientes_resultantes'] ='No Parientes'
df_no_parientes_resultante = pd.pivot_table(df_no_parientes,
                                            index = 'id_hog',
                                            columns = "No_Parientes_resultantes",
                                            values = "Cuenta",
                                            aggfunc = "sum")

In [19]:
df_no_parientes_resultante

No_Parientes_resultantes,No Parientes
id_hog,
41,1


# 2 Implementacion de la funcion  tipologia de hogar en la Data censo 2010

#### 2.1 Carga de data

In [20]:
path_censo = 'base_censo_2010/CPV2010m_Persona_filtro_variables_tipo_hogar.sav'

In [21]:
df_censo_2010, metadata_censo_2010 = pyreadstat.read_sav(path_censo, apply_value_formats=False)

In [22]:
df_censo_2010

,P01,P02,id_hog
0,1.0,1.0,010150001001010011
1,2.0,2.0,010150001001010011
2,2.0,3.0,010150001001010011
3,2.0,3.0,010150001001010011
4,2.0,3.0,010150001001010011
...,...,...,...
14483494,1.0,1.0,900451999022040031
14483495,2.0,2.0,900451999022040031
14483496,1.0,3.0,900451999022040031
14483497,2.0,3.0,900451999022040031


In [23]:
print(df_censo_2010.shape)
df_censo_2010.head()

(14483499, 3)


,P01,P02,id_hog
0,1.0,1.0,010150001001010011
1,2.0,2.0,010150001001010011
2,2.0,3.0,010150001001010011
3,2.0,3.0,010150001001010011
4,2.0,3.0,010150001001010011


In [24]:
df_censo_2010['id']=df_censo_2010['id_hog']

In [25]:
# lista de columnas del DataFrame
df_censo_2010.columns

Index(['P01', 'P02', 'id_hog', 'id'], dtype='object')

#### 2.2 Preparar la data con groupby

In [26]:
# Conversion de tipo de datos string
df_censo_2010['P02'] = df_censo_2010['P02'].astype(str)
df_censo_2010['P01'] = df_censo_2010['P01'].astype(str)
df_censo_2010['id_hog'] = df_censo_2010['id_hog'].astype(str)

In [27]:
# Agrupamiento por identificador de hogar y relacion con jefe de hogar
data_agrupada_censo = df_censo_2010.groupby(["id_hog", 'P02'])

In [28]:
data_agrupada_censo.size()

id_hog              P02
010150001001010011  1.0    1
                    2.0    1
                    3.0    3
010150001001010021  1.0    1
                    3.0    1
                          ..
900451999022040021  3.0    4
                    7.0    1
900451999022040031  1.0    1
                    2.0    1
                    3.0    3
Length: 10394557, dtype: int64

In [29]:
df_conteo_censo = data_agrupada_censo.size()

In [30]:
df_resultante_censo = df_conteo_censo.reset_index(name='Cuenta de parentesco o relación tiene con el/la jefe/a del hogar ')

In [31]:
%%time
df_resultante_censo["Codigo Hogar"] = df_resultante_censo.groupby(["id_hog"])['P02'].transform(lambda x : ''.join(x))

Wall time: 12min 18s


In [32]:
df_resultante_censo

,id_hog,P02,Cuenta de parentesco o relación tiene con el/la jefe/a del hogar,Codigo Hogar
0,010150001001010011,1.0,1,1.02.03.0
1,010150001001010011,2.0,1,1.02.03.0
2,010150001001010011,3.0,3,1.02.03.0
3,010150001001010021,1.0,1,1.03.0
4,010150001001010021,3.0,1,1.03.0
...,...,...,...,...
10394552,900451999022040021,3.0,4,1.02.03.07.0
10394553,900451999022040021,7.0,1,1.02.03.07.0
10394554,900451999022040031,1.0,1,1.02.03.0
10394555,900451999022040031,2.0,1,1.02.03.0


In [33]:
df_tipo_hogar = df_resultante_censo[df_resultante_censo['P02']=='1.0']

In [34]:
df_tipo_hogar

,id_hog,P02,Cuenta de parentesco o relación tiene con el/la jefe/a del hogar,Codigo Hogar
0,010150001001010011,1.0,1,1.02.03.0
3,010150001001010021,1.0,1,1.03.0
5,010150001001010031,1.0,1,1.03.05.0
8,010150001001010041,1.0,1,1.02.03.05.08.0
13,010150001001010051,1.0,1,1.0
...,...,...,...,...
10394543,900451999022030021,1.0,1,1.0
10394544,900451999022030131,1.0,1,1.02.03.0
10394547,900451999022040011,1.0,1,1.02.03.0
10394550,900451999022040021,1.0,1,1.02.03.07.0


In [35]:
df_diccionario_hogar = df_tipo_hogar[['id_hog','Codigo Hogar']]

In [36]:
df_diccionario_hogar

,id_hog,Codigo Hogar
0,010150001001010011,1.02.03.0
3,010150001001010021,1.03.0
5,010150001001010031,1.03.05.0
8,010150001001010041,1.02.03.05.08.0
13,010150001001010051,1.0
...,...,...
10394543,900451999022030021,1.0
10394544,900451999022030131,1.02.03.0
10394547,900451999022040011,1.02.03.0
10394550,900451999022040021,1.02.03.07.0


In [37]:
df_diccionario_hogar['Codigo Hogar'].value_counts()

1.02.03.0                1368542
1.03.0                    477264
1.0                       459610
1.02.0                    276403
1.02.03.07.0              144548
                          ...   
1.03.04.07.08.09.0             1
1.02.04.07.08.09.0             1
1.02.05.06.07.08.09.0          1
1.03.04.06.08.09.0             1
1.02.04.05.08.09.0             1
Name: Codigo Hogar, Length: 227, dtype: int64

In [38]:
df_diccionario_hogar['id']=df_diccionario_hogar['id_hog']

C:\Users\Dell\AppData\Local\Temp/ipykernel_432/4225452957.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_diccionario_hogar['id']=df_diccionario_hogar['id_hog']


In [39]:
df_diccionario_hogar

,id_hog,Codigo Hogar,id
0,010150001001010011,1.02.03.0,010150001001010011
3,010150001001010021,1.03.0,010150001001010021
5,010150001001010031,1.03.05.0,010150001001010031
8,010150001001010041,1.02.03.05.08.0,010150001001010041
13,010150001001010051,1.0,010150001001010051
...,...,...,...
10394543,900451999022030021,1.0,900451999022030021
10394544,900451999022030131,1.02.03.0,900451999022030131
10394547,900451999022040011,1.02.03.0,900451999022040011
10394550,900451999022040021,1.02.03.07.0,900451999022040021


In [40]:
#Guardar con libreria pyredstat
#df_diccionario_hogar.to_csv('df_diccionario_hogar.csv', index=False)

In [41]:
%%time
# Agrupamiento por identificador de hogar y sexo 
data_hijos_censo = df_censo_2010.groupby(["id_hog",'P01','P02'])
# tamaño de cada tipo de categoria en variable P01
df_conteo_hijos_censo = data_hijos_censo.size()
df_hijos_censo = df_conteo_hijos_censo.reset_index(name='Cuenta genero')
df_genero_hijos_censo= df_hijos_censo[(df_hijos_censo['P02']=='3.0')]
df_conteo_hijos_resultante_censo=pd.pivot_table(df_genero_hijos_censo,
                                                index='id_hog',
                                                columns='P01',
                                                values='Cuenta genero')
df_conteo_hijos_resultante_censo['id']=df_conteo_hijos_resultante_censo.index
df_conteo_hijos_resultante_censo

Wall time: 45.2 s


P01,1.0,2.0,id
id_hog,,,
010150001001010011,NaN,3.0,010150001001010011
010150001001010021,1.0,NaN,010150001001010021
010150001001010031,3.0,2.0,010150001001010031
010150001001010041,2.0,3.0,010150001001010041
010150001001010071,1.0,1.0,010150001001010071
...,...,...,...
900451999022020081,1.0,NaN,900451999022020081
900451999022030131,NaN,2.0,900451999022030131
900451999022040011,1.0,2.0,900451999022040011


In [42]:
%%time
data_parientes_censo = df_censo_2010.groupby(["id_hog",'P02'])
conteo_parientes_censo = data_parientes_censo.size()
df_parientes_censo =conteo_parientes_censo.reset_index(name='Cuenta')
df_parientes_censo = df_parientes_censo[(df_parientes_censo['P02']=='4.0')|(df_parientes_censo['P02']=='5.0')|(df_parientes_censo['P02']=='6.0')|(df_parientes_censo['P02']=='7.0')]
df_parientes_censo['Parientes_resultantes'] ='Parientes'
df_parientes_resultante_censo = pd.pivot_table(df_parientes_censo,
                                               index = 'id_hog',
                                               columns = "Parientes_resultantes" ,
                                               values = "Cuenta",
                                               aggfunc = "sum")
df_parientes_resultante_censo['id']=df_parientes_resultante_censo.index
df_parientes_resultante_censo

Wall time: 18.9 s


Parientes_resultantes,Parientes,id
id_hog,,
010150001001010031,2,010150001001010031
010150001001010041,1,010150001001010041
010150001001010081,3,010150001001010081
010150001001010131,1,010150001001010131
010150001001010171,1,010150001001010171
...,...,...
900451999021030591,4,900451999021030591
900451999021030631,1,900451999021030631
900451999021030671,6,900451999021030671


In [43]:
%%time
data_no_parientes_censo = df_censo_2010.groupby(["id_hog",'P02'])
conteo_no_parientes_censo = data_no_parientes_censo.size()
df_no_parientes_censo = conteo_no_parientes_censo.reset_index(name='Cuenta')
df_no_parientes_censo = df_no_parientes_censo[(df_no_parientes_censo['P02']=='8.0')|(df_no_parientes_censo['P02']=='9.0')|(df_no_parientes_censo['P02']=='10.0')|(df_no_parientes_censo['P02']=='11.0')]
df_no_parientes_censo['No_Parientes_resultantes'] ='No Parientes'
df_no_parientes_resultante_censo = pd.pivot_table(df_no_parientes_censo,
                                                    index = 'id_hog',
                                                    columns = "No_Parientes_resultantes",
                                                    values = "Cuenta",
                                                    aggfunc = "sum")
df_no_parientes_resultante_censo['id']=df_no_parientes_resultante_censo.index
df_no_parientes_resultante_censo

Wall time: 14.2 s


No_Parientes_resultantes,No Parientes,id
id_hog,,
010150001001010041,1,010150001001010041
010150001001010481,2,010150001001010481
010150001001020281,1,010150001001020281
010150001001020311,1,010150001001020311
010150001001030081,1,010150001001030081
...,...,...
900451999019020171,1,900451999019020171
900451999020020101,1,900451999020020101
900451999021010081,2,900451999021010081


# Merge de tipologia de hogar, numero de hijos e hijas, parientes y no parientes

In [44]:
df_censo_2010.head()

,P01,P02,id_hog,id
0,1.0,1.0,010150001001010011,010150001001010011
1,2.0,2.0,010150001001010011,010150001001010011
2,2.0,3.0,010150001001010011,010150001001010011
3,2.0,3.0,010150001001010011,010150001001010011
4,2.0,3.0,010150001001010011,010150001001010011


In [45]:
%%time
df_result1 = pd.merge(df_censo_2010, df_conteo_hijos_resultante_censo, on= 'id', how='outer' )

Wall time: 24.3 s


In [46]:
df_result1

,P01,P02,id_hog,id,1.0,2.0
0,1.0,1.0,010150001001010011,010150001001010011,NaN,3.0
1,2.0,2.0,010150001001010011,010150001001010011,NaN,3.0
2,2.0,3.0,010150001001010011,010150001001010011,NaN,3.0
3,2.0,3.0,010150001001010011,010150001001010011,NaN,3.0
4,2.0,3.0,010150001001010011,010150001001010011,NaN,3.0
...,...,...,...,...,...,...
14483494,1.0,1.0,900451999022040031,900451999022040031,1.0,2.0
14483495,2.0,2.0,900451999022040031,900451999022040031,1.0,2.0
14483496,1.0,3.0,900451999022040031,900451999022040031,1.0,2.0
14483497,2.0,3.0,900451999022040031,900451999022040031,1.0,2.0


In [47]:
%%time
df_result2 = pd.merge(df_result1, df_parientes_resultante_censo, on='id', how='outer')

Wall time: 26.5 s


In [48]:
df_result2

,P01,P02,id_hog,id,1.0,2.0,Parientes
0,1.0,1.0,010150001001010011,010150001001010011,NaN,3.0,NaN
1,2.0,2.0,010150001001010011,010150001001010011,NaN,3.0,NaN
2,2.0,3.0,010150001001010011,010150001001010011,NaN,3.0,NaN
3,2.0,3.0,010150001001010011,010150001001010011,NaN,3.0,NaN
4,2.0,3.0,010150001001010011,010150001001010011,NaN,3.0,NaN
...,...,...,...,...,...,...,...
14483494,1.0,1.0,900451999022040031,900451999022040031,1.0,2.0,NaN
14483495,2.0,2.0,900451999022040031,900451999022040031,1.0,2.0,NaN
14483496,1.0,3.0,900451999022040031,900451999022040031,1.0,2.0,NaN
14483497,2.0,3.0,900451999022040031,900451999022040031,1.0,2.0,NaN


In [49]:
%%time
df_result3 = pd.merge(df_result2, df_no_parientes_resultante_censo,  on= 'id', how='outer')

Wall time: 38 s


In [50]:
df_result3

,P01,P02,id_hog,id,1.0,2.0,Parientes,No Parientes
0,1.0,1.0,010150001001010011,010150001001010011,NaN,3.0,NaN,NaN
1,2.0,2.0,010150001001010011,010150001001010011,NaN,3.0,NaN,NaN
2,2.0,3.0,010150001001010011,010150001001010011,NaN,3.0,NaN,NaN
3,2.0,3.0,010150001001010011,010150001001010011,NaN,3.0,NaN,NaN
4,2.0,3.0,010150001001010011,010150001001010011,NaN,3.0,NaN,NaN
...,...,...,...,...,...,...,...,...
14483494,1.0,1.0,900451999022040031,900451999022040031,1.0,2.0,NaN,NaN
14483495,2.0,2.0,900451999022040031,900451999022040031,1.0,2.0,NaN,NaN
14483496,1.0,3.0,900451999022040031,900451999022040031,1.0,2.0,NaN,NaN
14483497,2.0,3.0,900451999022040031,900451999022040031,1.0,2.0,NaN,NaN


In [ ]:
%%time
df_result4 = pd.merge(df_result3, df_diccionario_hogar,  on= 'id', how='outer')

In [ ]:
df_result4

In [ ]:
%%time
df_result_final = df_result4[df_result4['P02']=='1.0']

In [ ]:
df_result_final

In [ ]:
del df_result_final['id_hog_x']
del df_result_final['id_hog_y']

In [ ]:
df_result_final

In [ ]:
df_result_final = df_result_final.rename(columns={'1.0':'Numero_de_hijos',
                                                  '2.0':'Numero_de_hijas',
                                                  'id':'id_hog',
                                                  'No Parientes':'No_Parientes',
                                                  'Codigo Hogar':'Codigo_Hogar',                                                  
                                                 })

In [ ]:
df_result_final

In [ ]:
#df_result_final.fillna(0)

In [60]:
df_result_final.to_csv('df_result_final.csv', index=False)

In [61]:
pyreadstat.write_sav(df_result_final, "tipologia_hogar.sav")